# Análisis de sueldos con Python

## Data

* [2021.2 - sysarmy - Encuesta de remuneración salarial Argentina
](https://docs.google.com/spreadsheets/d/1-ZRznwS4TK74o90aOiCKS5SiXxUQ2buN1wxZIMHQmzQ/edit#gid=557755710) 
* [2021.2 - sysarmy - Encuesta de remuneración salarial Latam
](https://docs.google.com/spreadsheets/d/1BkBNt1MHVS7DeIlpgmK9l6krtSQ5t_olRhlcyxMeKy0/edit#gid=557755710)

1. Descargar, leer y unir en una sola estructura de datos. ¿Qué estructura elegiste y por qué?

In [ ]:
# En este primer punto se traerán los datos de los dos archivos para trabajar.
# Se ordenarán de tal forma que simplifique el trabajo posterior, así como la normalizacion de sus valores.

In [ ]:
# A continuación la importación de la encuesta de Latam y la definición de su estructura que será listas de diccionarios.

In [1]:
import csv

list_keys = []
list_values = []
list_dict_info_latam = []                                     # acá se guarda la info de los dict de Latam

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Latam - Latam.tsv', encoding="utf8") as s_latam:
    latam_reader = csv.reader(s_latam, delimiter='\t')
    for index, line in enumerate(latam_reader):
        #if index > 9:
            #print(f'{latam_line}')
        # Líneas 0 a 9 vacías
        # Líneas 10  -> preguntas
        # Líneas 11+ -> respuestas
        if index == 10:
            list_keys = line
            list_keys[0] = 'País'                              # La pregunta Estoy trabajando en se pasa a llamar País
            dict_base = dict()
            for key in list_keys:
                dict_base[key] = ''
            #print(line)
        elif index >= 11:
            list_values = line
            dict_info = dict_base.copy()
            for i, key in enumerate(dict_info.keys()):
                dict_info[key] = list_values[i]
            #print(dict_info)
            dict_info['Ciudad'] = dict_info[dict_info['País']]  # Se agrega una pregunta Ciudad con respuesta variable
            
            for i, key in enumerate(list_keys[1:19]):           # Para borrar las columnas de los países con sus regiones
                dict_info.pop(key)
            #print(dict_info)
            
            list_dict_info_latam.append(dict_info)
        



In [ ]:
# A continuación la importación de la encuesta de Argentina y la definición de la estructura listas de diccionario.

In [2]:
import csv

list_keys = []
list_values = []
list_dict_info_arg = []                           # Acá se guarda la info de los dict de Argentina

with open('2021.2 - sysarmy - Encuesta de remuneración salarial Argentina - Argentina.tsv', encoding="utf8") as s_arg:
    arg_reader = csv.reader(s_arg, delimiter='\t')
    for index, line in enumerate(arg_reader):
        #if index > 8:
            #print(f'{arg_line}')
        # Líneas 0 a 8 vacías
        # Líneas 9   -> preguntas
        # Líneas 10+ -> respuestas
        if index == 9:
            list_keys = line
            list_keys[0] = 'Ciudad'                #La pregunta Dónde estás trabajando se pasa a llamar Ciudad
            dict_base = dict()
            for key in list_keys:
                dict_base[key] = ''
            #print(line)
        elif index >= 10:
            list_values = line
            dict_info = dict_base.copy()
            for i, key in enumerate(dict_info.keys()):
                dict_info[key] = list_values[i]
            #print(dict_info)
            dict_info['País'] = 'Argentina'          # Se agrega una pregunta País con respuesta Argentina
            
            list_dict_info_arg.append(dict_info)
                

In [ ]:
# A continuación seran unidas las dos estructuras en una sola. Argentina se agregará a Latam

In [3]:
list_all = list_dict_info_latam + list_dict_info_arg

In [4]:
print(len(list_all))

6869


In [5]:
print(len(list_dict_info_latam))

429


In [6]:
print(len(list_dict_info_arg))

6440


In [ ]:
# A continuación se normalizarán los elementos

In [8]:
# Función de normalización:

def normalize_record(dict_record):
    dict_result = {}
    
    for i, key in enumerate(dict_record.keys()):
        if key == 'Ciudad':
            if dict_record[key] == '':
                dict_result[key] = 'SIN CIUDAD'
            else:
                dict_result[key] = dict_record[key]
        elif key == '¿Contribuís a proyectos open source?' or key == '¿Programás como hobbie?':
            dict_result[key] = normalize_yes_no_empty(dict_record[key])
        elif key == 'Años de experiencia':
            #print(dict_record[key])
            dict_result[key] = normalize_FLOAT(dict_record[key])
        elif key == 'Años en la empresa actual' or key == 'Años en el puesto actual':
            dict_result[key] = normalize_INT(dict_record[key])
        elif key == 'Carrera':
            dict_result[key] = normalize_carrera(dict_record[key])
        else:
            dict_result[key] = dict_record[key]
      
    return dict_result

list_all_normalized = []
for i, key in enumerate(list_all):
    list_all_normalized.append(normalize_record(list_all[i]))
    
#print(list_all_normalized[0].keys())
register_total = len(list_all_normalized)

#Para verificar que se cambió bien el registro    
print(list_all[0])
print('---------------------------------------------------------------------------------')
print(list_all_normalized[0])
    

{'País': 'Bolivia', 'Tipo de contrato': 'Full-Time', 'Salario mensual o retiro BRUTO (en tu moneda local)': '13500', 'Salario mensual o retiro NETO (en tu moneda local)': '12000', 'Pagos en dólares': '', '¿Cuál fue el último valor de dólar que tomaron?': '', '¿Qué tan conforme estás con tu sueldo?': '3', 'Cómo creés que está tu sueldo con respecto al último semestre': '3', 'Recibís algún tipo de bono': 'Menos de un sueldo', 'A qué está atado el bono': 'Performance de la compañía', '¿Tuviste ajustes por inflación durante 2021?': 'No', '¿De qué % fue el ajuste total?': '0', '¿En qué mes fue el último ajuste?': '0', 'Trabajo de': 'SysAdmin / DevOps / SRE', 'Años de experiencia': '6', 'Años en la empresa actual': '14', 'Años en el puesto actual': '6', '¿Gente a cargo?': '2', 'Plataformas': 'Amazon Web Services, Azure, Docker, Linux, OpenStack', 'Lenguajes de programación o tecnologías.': 'Bash/Shell', 'Frameworks, herramientas y librerías': 'Ninguno de los anteriores', 'Bases de datos': 'A

In [ ]:
#Definiciones de funciones a utilizar

In [20]:
# Función que devuelve el porcentaje

def percentage(total,quantity):
    return round(quantity * 100  / total, 2)


# Función que calcula la paticpación ordenada descendiente de una clave

def participation_per_key_DESC(list_to_analyze, key):
    dict_result_Participacion = {}

    for record in list_to_analyze:
        if record[key] != '':
            if record[key] in dict_result_Participacion:
                # Si la ciudad ya esta en el diccionario resultado incremento el contador
                dict_result_Participacion[record[key]] += 1 
            else:
                # Si la ciudad NO esta en el diccionario resultado inicializo el contador en 1
                dict_result_Participacion[record[key]] = 1   

    return sorted(dict_result_Participacion.items(), key=lambda x: x[1], reverse=True)

# Función que calcula la particpación ordenada descendiente de dos claves

def participation_per_keys_Order_by_Keys(list_to_analyze, key1, key2):
    dict_result_Participacion = {}

    for record in list_to_analyze:
        if record[key1] != '':
            if (record[key1], record[key2]) in dict_result_Participacion:
                # Si la clave ya esta en el diccionario resultado, incremento el contador
                dict_result_Participacion[(record[key1], record[key2])] += 1 
            else:
                # Si la clave NO esta en el diccionario resultado, inicializo el contador en 1
                dict_result_Participacion[(record[key1], record[key2])] = 1   

    return sorted(dict_result_Participacion.items(), key=lambda x: x[0], reverse=False)

# Función para calcular la suma de las cantidades de una lista de tuplas (clave, cantidad)

def participation_count(tuple_list):
    total_count = 0
    for tuple_key_participation in tuple_list:
        total_count += tuple_key_participation[1]
    return total_count
    
# Función normalización Si No No contesta

def normalize_yes_no_empty(value):
    if value == '':
        normalized_value = 'No contesta'
    else:
        normalized_value = value
        
    return normalize_STR(normalized_value)

# Probar funcion normalize_yes_no_empty
#normalize_yes_no_empty('')

# Función para normalizar STR

def normalize_STR(value):
    normalized_value = str(value).strip().capitalize()
    normalized_value = normalized_value.replace('á', 'a')
    normalized_value = normalized_value.replace('é', 'e')
    normalized_value = normalized_value.replace('í', 'i')
    normalized_value = normalized_value.replace('ó', 'o')
    normalized_value = normalized_value.replace('ú', 'u')
    normalized_value = normalized_value.replace('ñ', 'n')
    return normalized_value

# Probar funcion normalize_STR
#normalize_STR('agAáéíóúSgiaASs')

# Funcion para normalizar calores a Float

def normalize_FLOAT(value):
    normalized_value = round(float(value),2)
    return normalized_value

# Probar funcion normalize_FLOAT
#normalize_FLOAT('1.1123123')

# Función para normalizar valores a INT (de STR a INT) 

def normalize_INT(value):
    normalized_value = int(normalize_FLOAT(value))
    return normalized_value

# Probar funcion normalize_INT
#normalize_INT('19.0')


#Función para normalizar la columna Carrera

def normalize_carrera(value):
    normalized_value = str(value).strip().lower()
    if normalized_value in ['.', '-', 'x', 'no', 'cp', 'n/a', 'no estudie', 'ninguna', 'ninguna.', 'ninguna de las anteriores']:
        normalized_value = 'Otros'
    normalized_value = replace_string_start(normalized_value, 'lic.', 'Licenciatura')
    normalized_value = replace_string_start(normalized_value, 'lic', 'Licenciatura')
    #normalized_value = normalized_value.replace('tec.', 'Tecnicatura')
    #normalized_value = normalized_value.replace('tec', 'Tecnicatura')
    #normalized_value = normalized_value.replace('cs.', 'Ciencias')
    #normalized_value = normalized_value.replace('cs', 'Ciencias')
    #normalized_value = normalized_value.replace('ed.', 'Educacion')
    #normalized_value = normalized_value.replace('ed', 'Educacion')
    #normalized_value = normalized_value.replace('cm', 'Community manager')
    #normalized_value = normalized_value.replace('rrii', 'Relaciones Internacionales')
    #normalized_value = normalized_value.replace('rrhh', 'Recursos humanos')
    #normalized_value = normalized_value.replace('arq', 'Arquitecto')
    #normalized_value = normalized_value.replace('ing.', 'Ingeniero')
    #normalized_value = normalized_value.replace('ing', 'Ingeniero')
    #normalized_value = normalized_value.replace('mg.', 'Magister')
    #normalized_value = normalize_STR(normalized_value)
    return normalized_value


# Funcion que remplaza un string solo si esta al inicio del mismo
def replace_string_start(a_string, replace_this, new_string):
    result = a_string
    if a_string[:len(replace_this)] == replace_this:
        result = new_string + a_string[len(replace_this):]
    return result

print(replace_string_start('lic test','lic.','Licenciatura'))

# Probar funcion normalize_carrera
normalize_carrera('lic. VhnkYghs')

lic test


'Licenciatura vhnkyghs'

2. Printear porcentaje de participación por región, ordenado de mayor a menor. Ej: 

```
- Ciudad Autónoma de Buenos Aires - 59.4%
- Formosa - 0.04%
```

In [ ]:
sorted_tuples_Participacion_Region = participation_per_key_DESC(list_all_normalized, 'Ciudad')
register_total = participation_count(sorted_tuples_Participacion_Region)

for tuple_record in sorted_tuples_Participacion_Region:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


In [ ]:
print(sorted_tuples_Participacion_Region)

3. Printear porcentaje de participación por tipo de rol, ordenado de mayor a menor. Ej:  
`- Developer - 39.11 %`

In [ ]:
sorted_tuples_Participacion_Rol = participation_per_key_DESC(list_all_normalized, 'Trabajo de')
register_total = participation_count(sorted_tuples_Participacion_Rol)

for tuple_record in sorted_tuples_Participacion_Rol:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')


4. Printear porcentaje de respuesta para las preguntas: 
* ¿Contribuís a proyectos Open Source?
* ¿Programás por hobbie?

In [ ]:
sorted_tuples_Participacion_Open_Source = participation_per_key_DESC(list_all_normalized, '¿Contribuís a proyectos open source?')
register_total = participation_count(sorted_tuples_Participacion_Open_Source)

print('¿Contribuís a proyectos Open Source?')
for tuple_record in sorted_tuples_Participacion_Open_Source:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

In [ ]:
sorted_tuples_Participacion_Hobbie = participation_per_key_DESC(list_all_normalized, '¿Programás como hobbie?')
register_total = participation_count(sorted_tuples_Participacion_Hobbie)

print('¿Programás por hobbie?')
for tuple_record in sorted_tuples_Participacion_Hobbie:
    print(f' - {tuple_record[0]} - {percentage(register_total,tuple_record[1])}%')

5. Printear porcentaje por seniority según años de experiencia, el mapeo es:
```
Junior: de 0 hasta 2 años.
Semi-Senior: de 2 años inclusive hasta 5 años.
Senior: desde 5 años inclusive.
``` 

In [ ]:
dict_Seniority_Years = {'Junior': 0, 'Semi-Senior': 0, 'Senior': 0}

for dict_line in list_all_normalized:
    years_of_experiencie = dict_line['Años de experiencia']
    if years_of_experiencie < 2:
        dict_Seniority_Years['Junior'] += 1
    elif years_of_experiencie >= 5:
        dict_Seniority_Years['Senior'] += 1
    else:
        dict_Seniority_Years['Semi-Senior'] += 1
    
#print(dict_Seniority_Years)

register_total = dict_Seniority_Years['Junior'] + dict_Seniority_Years['Senior'] + dict_Seniority_Years['Semi-Senior']

print('Porcentaje de participación por seniority:')
for key in dict_Seniority_Years.keys():
    print(f' {key} - {percentage(register_total,dict_Seniority_Years[key])}%')



6. Printear porcentaje de personas encuestadas por años en la compañía actual y por años en el puesto actual.

In [ ]:
sorted_tuples_Participacion_Anos_Empresa_Actual = participation_per_key_DESC(list_all_normalized, 'Años en la empresa actual')
register_total = participation_count(sorted_tuples_Participacion_Anos_Empresa_Actual)

print('Porcentaje de personas encuestadas por años en la compañia actual:')
for tuple_record in sorted_tuples_Participacion_Anos_Empresa_Actual:
    print(f' - {normalize_INT(tuple_record[0])} años - {percentage(register_total,tuple_record[1])}%')

In [ ]:
#Se detectaron dos valores fuera del rango bla bla bla 

In [ ]:
sorted_tuples_Participacion_Anos_Puesto_Actual = participation_per_key_DESC(list_all_normalized, 'Años en el puesto actual')
register_total = participation_count(sorted_tuples_Participacion_Anos_Puesto_Actual)

print('Porcentaje de personas encuestadas por años en su puesto actual:')
for tuple_record in sorted_tuples_Participacion_Anos_Puesto_Actual:
    print(f' - {normalize_INT(tuple_record[0])} años - {percentage(register_total,tuple_record[1])}%')

7. Printear porcentajes de nivel de educación formal y estado, es decir: % educación secundaria, terciaria, universitaria, postgrado, doctorado, postdoctorado, completo, incompleto y en curso para cada uno.

In [ ]:
sorted_tuples_Participacion_Nivel_Estudios = participation_per_keys_Order_by_Keys(list_all_normalized, 'Nivel de estudios alcanzado', 'Estado')
#print(sorted_tuples_Participacion_Nivel_Estudios)
register_total_tuples_Participacion_Nivel_Estudios = participation_count(sorted_tuples_Participacion_Nivel_Estudios)
   
print('Porcentaje de nivel de educación formal y su estado:')
for tuple_record in sorted_tuples_Participacion_Nivel_Estudios:
    print(f' - {tuple_record[0][0]} {tuple_record[0][1]} - {percentage(register_total_tuples_Participacion_Nivel_Estudios,tuple_record[1])}%')
    


8. Formatear las carreras universitarias:
- Nombres capitalizados
- Reemplazar vocales con tilde por vocales sin tilde.
- Reemplazar `ñ` por `n`
- *lic*, *lic.* por Licenciatura
- *tec*, *tec.* por Tecnicatura
* *cs, *cs.* por Ciencias
* *ed, ed.* por Educación
* Transformaciones que se consideren necesarias

Printear porcentaje según carrera

In [9]:
sorted_tuples_Participacion_Carrera = participation_per_key_DESC(list_all_normalized, 'Carrera')
register_total = participation_count(sorted_tuples_Participacion_Carrera)

print('Porcentaje de participación por carrera:')
for tuple_record in sorted_tuples_Participacion_Carrera:
    print(f' - {(tuple_record[0])} - {percentage(register_total,tuple_record[1])}%')

Porcentaje de participación por carrera:
 - ingeniería en sistemas de información - 19.0%
 - ingeniería en informática - 11.34%
 - analista de sistemas - 8.68%
 - Licenciaturaenciatura en ciencias de la computación - 5.66%
 - Licenciaturaenciatura en sistemas de información - 5.66%
 - Licenciaturaenciatura en informática - 4.3%
 - tecnicatura en programación - 3.39%
 - ingeniería electrónica - 2.99%
 - ingeniería en computación - 2.78%
 - diseño gráfico - 2.48%
 - tecnicatura superior en programación - 2.36%
 - Licenciaturaenciatura en administración - 1.72%
 - tecnicatura en informática - 1.57%
 - ingeniería industrial - 1.18%
 - Licenciaturaenciatura en análisis de sistemas - 1.18%
 - analista programador - 1.12%
 - ingeniería en software - 1.06%
 - analista programador universitario - 1.0%
 - ingeniería en telecomunicaciones - 0.97%
 - analista de computación - 0.88%
 - tecnicatura superior en sistemas informáticos - 0.88%
 - diseño multimedial - 0.85%
 - tecnicatura en sistemas - 0

9. Printear porcentaje de respuesta a las preguntas:
- ¿Realizaste cursos de especialización?
- Si realizaste, ¿Quién pagó por los cursos de especialización?

10. Printear porcentaje por identidad de género y personas con discapacidad

11. Salarios: calcular la mediana salarial para cada una de las siguientes categorías:
- Salario según región
- Salario por rol
- Salario por experiencia
- Salario por nivel de formación
- Salario por carrera
- Salario por tecnología
- Salario por lenguaje de programación
- Salario por género

12. Printear porcentaje y salario según tipos de contrato .

13. En base a los resultados obtenidos confeccionar conclusiones respecto a:

- Rol vs sueldos
- Nivel de educación alcanzada vs sueldos
- Género vs sueldos
- Tecnologías populares

14. Generar gráficos de barra para mostrar los resultados mencionados anteriormente usando strings, ej:

```
developer | -----------------------------------------------
sysadmin  | ---------------------------
QA        | ------------------
          | ..................5%.......10%...............40%
```